In [1]:
#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, GeFGEFME3mrqG3SKi9oNK43ARBhaQMrTyj1qAQzxTqJB, 0>


# Create: Vertices/nodes, edges ..

In [6]:
import pandas as pd

print("--")

--


In [7]:
#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


,airport_code,airport_name,LABEL
0,MKE,Milwaukee,Airport
1,ORD,Chicago O-Hare,Airport
2,SJC,San Jose,Airport
3,DEN,Denver,Airport


In [8]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


,START_ID,END_ID,DISTANCE,NUM_HOPS,TYPE
0,MKE,ORD,66,1,FLIES_TO
1,ORD,MKE,66,1,FLIES_TO
2,ORD,DEN,886,1,FLIES_TO
3,DEN,ORD,886,1,FLIES_TO
4,SJC,DEN,948,1,FLIES_TO
5,DEN,SJC,948,1,FLIES_TO
6,SJC,ORD,1829,1,FLIES_TO


In [9]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [ ]:
# ##################################################################

In [10]:

#  Add a second node type, and related edge
#

l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [ ]:
# ##################################################################

In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:

#  Full graph

l_result = my_graph.query("""
   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   """, contextualize=True)


l_result.view()


#  The Jira as it is being reported 

In [ ]:

#  We will comare this cell to the next; no error here
#

l_query  = """
   MATCH(n)
   RETURN n
   """.format()

l_result = my_graph.query(l_query)

display(print(l_result))

#  Returns,
#
#          n.internal_id      n.labels n.restaurant_code            n.restaurant_name n.type  n.LABEL n.airport_code  n.airport_name
#     0                2  [Restaurant]               PAP  Pappadeauxs Seafood Kitchen   node      NaN            NaN             NaN
#     1                0     [Airport]               NaN                          NaN   node  Airport            MKE       Milwaukee
#     2                1     [Airport]               NaN                          NaN   node  Airport            ORD  Chicago O-Hare
#     3  281474976710656     [Airport]               NaN                          NaN   node  Airport            SJC        San Jose
#     4  281474976710657     [Airport]               NaN                          NaN   node  Airport            DEN          Denver


In [12]:

#  I do not receive a value for some of these fields
#
#  The Analytics folks make use of internal_id
#

l_query  = """
   MATCH (n) 
   RETURN 
   
      // n.internal_id     AS my_id,                            //  None
      n.id                 AS my_id,                            //  None
      
      n.labels             AS my_label,                         //  None
      n.airport_code       AS airport_code, 
      n.airport_name       AS airport_name, 
      n.restaurant_code    AS restaurant_code, 
      n.restaurant_name    AS restaurant_name,
      n.LABEL              AS my_label2                         //  Notice the value is present above for n.labels, missing here
   """.format()

l_return = my_graph.query(l_query)
   #
print(l_return)

#  Returns,
#
#       my_id my_label airport_code    airport_name restaurant_code              restaurant_name my_label2
#     0  None     None          ORD  Chicago O-Hare            None                         None   Airport
#     1  None     None          MKE       Milwaukee            None                         None   Airport
#     2  None     None         None            None             PAP  Pappadeauxs Seafood Kitchen      None
#     3  None     None          DEN          Denver            None                         None   Airport
#     4  None     None          SJC        San Jose            None                         None   Airport


          0/? [?op/s]

  my_id my_label airport_code    airport_name restaurant_code              restaurant_name my_label2
0  None     None          SJC        San Jose            None                         None   Airport
1  None     None         None            None             PAP  Pappadeauxs Seafood Kitchen      None
2  None     None          ORD  Chicago O-Hare            None                         None   Airport
3  None     None          DEN          Denver            None                         None   Airport
4  None     None          MKE       Milwaukee            None                         None   Airport


In [ ]:

#  Compare this cell to the next
#

l_query  = """
   MATCH (n) 
   RETURN 
      n.internal_id       ,
      n.labels            ,
      n.airport_code      ,
      n.airport_name      ,
      n.restaurant_code   ,
      n.restaurant_name   ,
      n.LABEL              
   """.format()

l_return = my_graph.query(l_query)
   #
for l_each in l_return.itertuples():
   print(l_each)

#  Returns,
#
Pandas(Index=0, _1=None, _2=None, _3='ORD', _4='Chicago O-Hare', _5=None, _6=None, _7='Airport')
Pandas(Index=1, _1=None, _2=None, _3='MKE', _4='Milwaukee', _5=None, _6=None, _7='Airport')
Pandas(Index=2, _1=None, _2=None, _3=None, _4=None, _5='PAP', _6='Pappadeauxs Seafood Kitchen', _7=None)
Pandas(Index=3, _1=None, _2=None, _3='SJC', _4='San Jose', _5=None, _6=None, _7='Airport')
Pandas(Index=4, _1=None, _2=None, _3='DEN', _4='Denver', _5=None, _6=None, _7='Airport')



In [ ]:

#  And compare this cell to the next
#

l_query  = """
   MATCH (n) 
   RETURN n
   """.format()

l_return = my_graph.query(l_query)
   #
for l_each in l_return.itertuples():
   print(l_each)

#  Returns,
#
#     Pandas(Index=0, _1=1, _2=['Airport'], _3='Airport', _4='ORD', _5='Chicago O-Hare', _6='node', _7=nan, _8=nan)
#     Pandas(Index=1, _1=2, _2=['Restaurant'], _3=nan, _4=nan, _5=nan, _6='node', _7='PAP', _8='Pappadeauxs Seafood Kitchen')
#     Pandas(Index=2, _1=0, _2=['Airport'], _3='Airport', _4='MKE', _5='Milwaukee', _6='node', _7=nan, _8=nan)
#     Pandas(Index=3, _1=281474976710656, _2=['Airport'], _3='Airport', _4='SJC', _5='San Jose', _6='node', _7=nan, _8=nan)
#     Pandas(Index=4, _1=281474976710657, _2=['Airport'], _3='Airport', _4='DEN', _5='Denver', _6='node', _7=nan, _8=nan)


In [21]:

#  I can refer to these values by ordinal positioning, but, this breaks as fields move, get added, other.
#

l_query  = """
   MATCH (n) 
   RETURN n
   """.format()

l_return = my_graph.query(l_query)
   #
for l_each in l_return.itertuples():
   print( str(l_each[0]) + "   " +  str(l_each[1]) + "   " + str(l_each[2]) )

#  Returns,
#
#     0   2   ['Restaurant']
#     1   1   ['Airport']
#     2   0   ['Airport']
#     3   281474976710657   ['Airport']
#     4   281474976710656   ['Airport']


          0/? [?op/s]

0   2   ['Restaurant']
1   1   ['Airport']
2   0   ['Airport']
3   281474976710657   ['Airport']
4   281474976710656   ['Airport']
